In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import json
import malaya
from tqdm import tqdm
from unidecode import unidecode

/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3386
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/dev/malaya/malaya/tokenizer.py:208: FutureWarning: Possible nested set at position 3904
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [3]:
import re

def cleaning(string):
    splitted = malaya.text.function.split_into_sentences(string)
    if not len(splitted):
        splitted = '. '.join([k.strip() for k in string.split('.') if len(k.strip())])
    if splitted[0][0] == '-':
        splitted[0] = splitted[0].replace('- ','')
    points = [f'{no + 1}. {s}' for no, s in enumerate(splitted)]
    points = ' '.join(points)
    return points

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from malaya.text.vectorizer import SkipGramCountVectorizer

stopwords = malaya.text.function.get_stopwords()
bow = CountVectorizer(
    ngram_range = (1, 4),
    stop_words = stopwords,
    lowercase = False,
)

stopwords = malaya.text.function.get_stopwords()
skip_bow = SkipGramCountVectorizer(
    ngram_range = (1, 4),
    stop_words = stopwords,
    lowercase = False,
    skip = 2
)

In [5]:
from glob import glob
from bs4 import BeautifulSoup

files = sorted(glob('*.nested.semisupervised'))
files

['berita-bisnes.json.nested.semisupervised',
 'berita-dunia.json.nested.semisupervised',
 'berita-hiburan.json.nested.semisupervised',
 'berita-malaysia.json.nested.semisupervised',
 'berita-politik.json.nested.semisupervised',
 'berita-sukan.json.nested.semisupervised',
 'berita-teknologi.json.nested.semisupervised',
 'gaya-hidup.json.nested.semisupervised']

In [6]:
before, after = [], []

count = 0
rejected = []
languages = []
accepted = []
para = []
malaysian_news = {
    'kosmo',
    'hmetro',
    'malaymail',
    'projekmm',
    'bharian',
    'utusan',
    'astroawani',
    'themalaysianinsight',
    'malaysiakini',
    'bernama'
}

def reject(data):
    if data['news'] in malaysian_news:
        return False
    if any([n in data['top-image'] for n in malaysian_news]):
        return False
    if any([n in data['url'] for n in malaysian_news]):
        return False
    if 'com.my' in data['top-image']:
        return False
    if data['language'] == 'malay':
        return False
    if 'Siaran Pers' in data['news']:
        return True
    if '.id' in data['news']:
        return True
    
    return True

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            soup = BeautifulSoup(data['r']['response']['articleBody'], "lxml")
            text = BeautifulSoup(soup.text, 'lxml').text
            text = re.sub(r'[ ]+', ' ', text).strip()
            if 'kindly register' in text.lower() or 'disabled in your browser' in text.lower():
                continue
            if len(text.split()) < 30:
                continue
            accepted.append(text)

9622it [00:05, 1790.89it/s]
7897it [00:04, 1701.46it/s]
9956it [00:06, 1430.57it/s]
8578it [00:05, 1662.30it/s]
9970it [00:06, 1596.62it/s]
9826it [00:05, 1750.73it/s]
972it [00:00, 1649.89it/s]
5342it [00:04, 1270.26it/s]


In [7]:
len(accepted)

62150

In [8]:
accepted[0]

'KUALA LUMPUR: Ringgit dijangka diniagakan pada paras 4.38 hingga 4.40 berbanding dolar Amerika Syarikat (AS) minggu depan dengan kecenderungan meningkat terutamanya jika pasaran pertukaran asing terus menghadapi kelembapan dalam kitaran peningkatan kadar faedah AS dengan barisan Kabinet baharu Malaysia menawarkan sokongan selanjutnya.\n\r\nPengarah Urusan SPI Asset Management Stephen Innes berkata ini berdasarkan situasi yang masih tenang dan rancangan China untuk meneruskan pembukaan semula negara itu.\n\r\n"Satu-satunya cabaran yang saya perhatikan adalah potensi pergolakan awam di China pada akhir minggu ini kerana jika kita mengingat kembali pergolakan di Hong Kong pada beberapa tahun lepas, ia sebenarnya memuncak pada hujung minggu," katanya kepada Bernama.\n \r\nBeliau berkata pengumuman barisan Kabinet baharu Malaysia malam tadi akan memberi laluan kepada pembentangan bajet negara manakala pasaran akan kukuh apabila terdapat elemen kestabilan politik.\n\r\nSementara itu, bagi p

In [9]:
rejected = {'saya', 'awak', 'kami', 'dia', 'mereka', 'anda', 'kita'}

tokenize = malaya.tokenizer.Tokenizer().tokenize

def headline(string, length = 300):
    splitted = malaya.text.function.split_into_sentences(string)
    splitted = [s for s in splitted if all([r not in s for r in rejected])]
    selected, index, results = '', 0, []
    while len(selected) <= length and index < len(splitted):
        selected += splitted[index]
        results.append(splitted[index])
        index += 1
    return ' '.join(results)

def reject(string):
    splitted = malaya.text.function.split_into_sentences(string)
    splitted = [s for s in splitted if not len(set(tokenize(s.lower())) & rejected)]
    return ' '.join(splitted)

In [10]:
s = 'IBU negara Korea Selatan, Seoul menduduki tempat ketujuh dalam senarai bandar raya paling mahal untuk didiami pada tahun ini, lapor agensi berita Yonhap.'

In [12]:
reject(accepted[0])

'KUALA LUMPUR: Ringgit dijangka diniagakan pada paras 4.38 hingga 4.40 berbanding dolar Amerika Syarikat (AS) minggu depan dengan kecenderungan meningkat terutamanya jika pasaran pertukaran asing terus menghadapi kelembapan dalam kitaran peningkatan kadar faedah AS dengan barisan Kabinet baharu Malaysia menawarkan sokongan selanjutnya. Pengarah Urusan SPI Asset Management Stephen Innes berkata ini berdasarkan situasi yang masih tenang dan rancangan China untuk meneruskan pembukaan semula negara itu. Beliau berkata pengumuman barisan Kabinet baharu Malaysia malam tadi akan memberi laluan kepada pembentangan bajet negara manakala pasaran akan kukuh apabila terdapat elemen kestabilan politik. Sementara itu, bagi perbandingan mingguan, ringgit mencatat paras yang tidak diraih sejak Jun tahun ini dengan diniagakan pada 4.3835/3925 berbanding dolar AS daripada 4.4795/4890 pada Jumaat. Unit tempatan diniagakan kebanyakannya tinggi berbanding kumpulan mata wang utama. Ia meningkat berbanding d

In [13]:
import random

In [14]:
keywords = malaya.keyword.extractive.rake(accepted[2], 
                                                      vectorizer = bow, 
                                                      top_k = random.randint(3, 10))
keywords = [k[1] for k in keywords]
keywords_rake = malaya.keyword.extractive.rake(accepted[2], atleast = 1,
                                          top_k = random.randint(3, 10))
keywords_rake = [k[1] for k in keywords_rake]
keywords, keywords_rake

(['kemampanan sektor TriBE Assembly',
  'TriBE Assembly Jumaat Sarawak',
  'legasi kemampanan sektor TriBE',
  'Industri Kreatif Seni Persembahan',
  'alam Kementerian Pelancongan Industri',
  'Pelancongan Industri Kreatif Seni',
  'Kementerian Pelancongan Industri Kreatif'],
 ['Ketua Pegawai Eksekutif BESarawak Amelia Roziman',
  'Seni Persembahan Sarawak pelan strategik jangka',
  'Seri Abdul Karim Rahman Hamzah',
  '3rd Business Events Tribal Meet',
  'mencapai Strategi Pembangunan Pasca COVID-19',
  'penggiat acara perniagaan global berkembang'])

In [15]:
months = {
    'january',
    'jan',
    'januari',
    'february',
    'feb',
    'februari',
    'march',
    'mac',
    'april',
    'apr',
    'may',
    'mei',
    'june',
    'jun',
    'july',
    'julai',
    'august',
    'ogos',
    'aug',
    'september',
    'sep',
    'october',
    'oktober',
    'oct',
    'november',
    'nov',
    'december',
    'disember',
    'dec',
    'utusan',
    'malaysiakini',
    'astroawani',
    'bernama',
    'com',
}

In [16]:
from malaya.text.rouge import postprocess_summary, filter_rouge, _get_word_ngrams, _rouge_clean, cal_rouge
from malaya.text.function import split_into_sentences

def filtering_rouge(summary, contents, start = 0.15, increment = 0.05, break_at = 120, n = 1):
    filtered = contents[:]
    i = 0
    while len(filtered.split()) > break_at:
        filtered = filter_rouge(summary, filtered, n = n, threshold = start + increment * i)
        i += 1
    return filtered

In [17]:
i = 2
keywords = malaya.keyword.extractive.rake(accepted[i], 
                                                  vectorizer = bow, 
                                                  top_k = 50)
keywords = [simple_cleaning(k[1]) for k in keywords if len(k[1].split()) > 1 and len(k[1]) > 10 \
                and len(set(k[1].lower().replace('-', '').split()) & months) == 0]
keywords = keywords[:random.randint(3, 10)]
keywords, 

(['kemampanan sektor TriBE Assembly',
  'TriBE Assembly Jumaat Sarawak',
  'legasi kemampanan sektor TriBE',
  'Industri Kreatif Seni Persembahan',
  'alam Kementerian Pelancongan Industri',
  'Pelancongan Industri Kreatif Seni',
  'Kementerian Pelancongan Industri Kreatif'],)

In [18]:
before, after = [], []
for i in tqdm(range(len(accepted))):
    h = reject(accepted[i])
    try:

        keywords = malaya.keyword.extractive.rake(h, vectorizer = bow, top_k = 100)
        keywords = [simple_cleaning(k[1]) for k in keywords if len(k[1].split()) > 1 and len(k[1]) > 10 \
                        and len(set(k[1].lower().replace('-', '').split()) & months) == 0]
        keywords = keywords[:random.randint(3, 10)]
        
        r = filtering_rouge('. '.join(keywords), h, n = 2, start = 0.01)
        
        if len(r.split()) > 20:
            before.append(keywords)
            after.append(r)
        
        keywords_rake = malaya.keyword.extractive.rake(h, top_k = 100)
        keywords_rake = [simple_cleaning(k[1]) for k in keywords_rake if len(k[1].split()) > 1 and len(k[1]) > 10 \
                        and len(set(k[1].lower().replace('-', '').split()) & months) == 0]
        keywords_rake = keywords_rake[:random.randint(3, 10)]
        r = filtering_rouge('. '.join(keywords_rake), h, n = 2, start = 0.01)
        
        if len(r.split()) > 20:
            before.append(keywords_rake)
            after.append(r)
        
    except Exception as e:
        # print(h, accepted[i]['text'])
        print(e)

 37%|█████████████▌                       | 22841/62150 [04:20<06:38, 98.65it/s]

empty vocabulary; perhaps the documents only contain stop words


 37%|█████████████▍                      | 23281/62150 [04:24<04:56, 130.89it/s]

empty vocabulary; perhaps the documents only contain stop words


 38%|█████████████▌                      | 23340/62150 [04:25<05:11, 124.58it/s]

...

 83%|██████████████████████████████▌      | 51350/62150 [09:42<02:00, 89.36it/s]

empty vocabulary; perhaps the documents only contain stop words


100%|███████████████████████████████████▉| 62100/62150 [11:49<00:00, 122.74it/s]

empty vocabulary; perhaps the documents only contain stop words


100%|█████████████████████████████████████| 62150/62150 [11:49<00:00, 87.59it/s]


In [19]:
len(before), len(after)

(121822, 121822)

In [21]:
before[101], after[101]

(['ringgit diniagakan kebanyakannya rendah',
  'rendah berbanding dolar AS',
  'diniagakan kebanyakannya rendah berbanding',
  'kebanyakannya rendah berbanding kumpulan',
  'rendah berbanding kumpulan mata',
  'Management Stephen Innes ringgit',
  'Sementara ringgit diniagakan kebanyakannya',
  'berikutan aktiviti pengambilan untung'],
 'KUALA LUMPUR: Ringgit kembali ditutup rendah hari ini berbanding dolar AS selepas kenaikan minggu lepas berikutan aktiviti pengambilan untung, kata seorang peniaga. Pada 6 petang, unit tempatan terus susut kepada 4.5060/5130 berbanding dolar AS daripada paras penutup 4.4795/4890 pada Jumaat minggu lepas. Pengarah Urusan SPI Asset Management Stephen Innes berkata ringgit kembali menyusut hari ini apabila kes COVID-19 terus melonjak di China, yang memberi kesan kepada pembukaan semula ekonomi serta membawa kepada prospek lebih lemah. Sementara itu, ringgit diniagakan kebanyakannya rendah berbanding kumpulan mata wang utama lain.')

In [22]:
with open('keywords-headline-astroawani.json', 'w') as fopen:
    json.dump({'before': before, 'after': after}, fopen)